### Data Acquisition & Initial Global Inspection

In [1]:
import pandas as pd
import os

In [2]:
# 1. SETUP PATHS
path_processed = "../../data/processed/Ecommerce-Logistics-And-Revenue-Optimization_processed/"
path_raw = "../../data/raw/"

# List of all files based on your screenshot
processed_files = [
    'orders.csv', 'order_items.csv', 'sellers.csv', 'products.csv', 
    'category_translation.csv', 'customers.csv', 'geolocation.csv', 
    'order_payments.csv', 'order_reviews.csv'
]
raw_files = [
    'olist_marketing_qualified_leads_dataset.csv', 
    'olist_closed_deals_dataset.csv'
]

# 2. LOAD ALL DATA INTO A DICTIONARY
dfs = {}

print("--- Loading Data ---")
for f in processed_files:
    name = f.replace('.csv', '')
    dfs[name] = pd.read_csv(os.path.join(path_processed, f))

for f in raw_files:
    name = f.replace('olist_', '').replace('_dataset.csv', '')
    dfs[name] = pd.read_csv(os.path.join(path_raw, f))
print("All 11 tables loaded successfully.\n")

# 3. GLOBAL SUMMARY TABLE
summary_data = []
for name, df in dfs.items():
    summary_data.append({
        "Table": name,
        "Rows": df.shape[0],
        "Cols": df.shape[1],
        "Missing Values": df.isnull().sum().sum(),
        "Duplicates": df.duplicated().sum()
    })

df_summary = pd.DataFrame(summary_data)
print("--- DATASET OVERVIEW ---")
print(df_summary)

# 4. INSPECT THE "BRIDGE" (The most important part)
# We want to see how many sellers from the Funnel exist in the Sales data
funnel_sellers = dfs['closed_deals']['seller_id'].unique()
sales_sellers = dfs['sellers']['seller_id'].unique()

bridge_count = len(set(funnel_sellers).intersection(set(sales_sellers)))

print("\n--- THE BRIDGE ANALYSIS ---")
print(f"Sellers in Marketing Funnel: {len(funnel_sellers)}")
print(f"Sellers in Project 1 Data: {len(sales_sellers)}")
print(f"MATCHING SELLERS (The Bridge): {bridge_count}")

# 5. CHECKING COLUMN NAMES FOR MERGING
print("\n--- KEY COLUMNS FOR JOINS ---")
print(f"Marketing Leads Columns: {dfs['marketing_qualified_leads'].columns.tolist()}")
print(f"Closed Deals Columns: {dfs['closed_deals'].columns.tolist()}")

--- Loading Data ---
All 11 tables loaded successfully.

--- DATASET OVERVIEW ---
                        Table    Rows  Cols  Missing Values  Duplicates
0                      orders   99441     8            4908           0
1                 order_items  112650     7               0           0
2                     sellers    3095     4               0           0
3                    products   32951    10            1838           0
4        category_translation      71     2               0           0
5                   customers   99441     5               0           0
6                 geolocation   19015     5               0           0
7              order_payments  103886     5               0           0
8               order_reviews   99224     7          145903           0
9   marketing_qualified_leads    8000     4              60           0
10               closed_deals     842    14            3300           0

--- THE BRIDGE ANALYSIS ---
Sellers in Marketing Funn

### Marketing Funnel Cleaning & Seller Bridge Isolation

In [3]:
# 1. MERGE THE MARKETING FUNNEL
# We use a left join to keep all Leads, even if they didn't close (useful for Conversion Rate analysis)
df_funnel = pd.merge(dfs['marketing_qualified_leads'], dfs['closed_deals'], on='mql_id', how='left')

# 2. CONVERT DATES
date_cols = ['first_contact_date', 'won_date']
for col in date_cols:
    df_funnel[col] = pd.to_datetime(df_funnel[col])

# 3. FILL MISSING VALUES
df_funnel['origin'] = df_funnel['origin'].fillna('unknown')
df_funnel['business_segment'] = df_funnel['business_segment'].fillna('unknown')
df_funnel['business_type'] = df_funnel['business_type'].fillna('unknown')

# 4. ISOLATE THE BRIDGE SELLERS
# Get the list of IDs that exist in both datasets
bridge_seller_ids = set(dfs['closed_deals']['seller_id']).intersection(set(dfs['sellers']['seller_id']))

# Create a flag in our funnel to identify these "Project 1 Matching" sellers
df_funnel['is_bridge_seller'] = df_funnel['seller_id'].isin(bridge_seller_ids)

# 5. EXPORT THE CLEANED FILES
# Note: Using your requested path
export_path = "../../data/processed/"

df_funnel.to_csv(f"{export_path}cleaned_marketing_funnel.csv", index=False)

print(f"Success! Cleaned funnel exported to: {export_path}")
print(f"Bridge Sellers Isolated: {len(bridge_seller_ids)}")

Success! Cleaned funnel exported to: ../../data/processed/
Bridge Sellers Isolated: 380


### Data Integrity Check & Revenue Attribution Validation

In [4]:
# 1. LOAD THE CLEANED FILE WE JUST CREATED
df_cleaned_funnel = pd.read_csv("../../data/processed/cleaned_marketing_funnel.csv")

print("--- CLEANING VALIDATION ---")
# Check 1: Nulls in critical marketing columns
print(f"Nulls in 'origin': {df_cleaned_funnel['origin'].isnull().sum()}")
print(f"Nulls in 'business_segment': {df_cleaned_funnel['business_segment'].isnull().sum()}")

# Check 2: Date Conversion Success
print(f"Date Type for 'won_date': {df_cleaned_funnel['won_date'].dtype}")

# Check 3: The Bridge Breakdown
bridge_subset = df_cleaned_funnel[df_cleaned_funnel['is_bridge_seller'] == True]
print(f"\n--- BRIDGE DATA HEALTH (380 Sellers) ---")
print(f"Unique Sellers: {bridge_subset['seller_id'].nunique()}")
print(f"Top 5 Marketing Origins:\n{bridge_subset['origin'].value_counts().head(5)}")
print(f"\nTop 5 Business Segments:\n{bridge_subset['business_segment'].value_counts().head(5)}")

# Check 4: Revenue Data availability (Previewing the join)
# Let's see if our Bridge Sellers have actual sales in the order_items table
sales_check = dfs['order_items'][dfs['order_items']['seller_id'].isin(bridge_subset['seller_id'])]
print(f"\nTotal Order Rows for our Bridge Sellers: {len(sales_check)}")
print(f"Total Revenue from Bridge Sellers: ${sales_check['price'].sum():,.2f}")

--- CLEANING VALIDATION ---
Nulls in 'origin': 0
Nulls in 'business_segment': 0
Date Type for 'won_date': object

--- BRIDGE DATA HEALTH (380 Sellers) ---
Unique Sellers: 380
Top 5 Marketing Origins:
origin
organic_search    113
paid_search       101
unknown            85
direct_traffic     31
social             31
Name: count, dtype: int64

Top 5 Business Segments:
business_segment
health_beauty                      45
household_utilities                44
home_decor                         44
construction_tools_house_garden    32
audio_video_electronics            31
Name: count, dtype: int64

Total Order Rows for our Bridge Sellers: 5044
Total Revenue from Bridge Sellers: $676,851.48
